## Recurrent Neural Networks

In this assignment, we will learn about recurrent neural networks. We will create an RNN and learn to classify text data.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import one_hot

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
yelp = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/yelp_labeled.csv', error_bad_lines=False)

b'Skipping line 281: expected 2 fields, saw 3\nSkipping line 290: expected 2 fields, saw 3\nSkipping line 296: expected 2 fields, saw 3\nSkipping line 322: expected 2 fields, saw 3\nSkipping line 373: expected 2 fields, saw 3\nSkipping line 417: expected 2 fields, saw 3\nSkipping line 427: expected 2 fields, saw 3\nSkipping line 429: expected 2 fields, saw 3\nSkipping line 577: expected 2 fields, saw 3\nSkipping line 578: expected 2 fields, saw 3\nSkipping line 611: expected 2 fields, saw 3\nSkipping line 677: expected 2 fields, saw 3\nSkipping line 771: expected 2 fields, saw 3\nSkipping line 930: expected 2 fields, saw 3\nSkipping line 979: expected 2 fields, saw 4\nSkipping line 980: expected 2 fields, saw 3\n'


In [5]:
yelp.head()

,text,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [6]:
yelp.shape

(976, 2)

In [7]:
# check target for class balance
yelp.sentiment.value_counts()

1    494
0    482
Name: sentiment, dtype: int64

We have loaded a Yelp review dataset above. A positive sentiment is classified as 1 and a negative sentiment is classified as 0. 

In [8]:
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def remove_stopwords(input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words)       

def stem_list(word_list):
    stemmed = []
    for word in word_list:
        stemmedword = stemmer.stem(word)
        stemmed.append(stemmedword)
    return stemmed

def normalize(terms):
    terms = terms.lower()
    terms = remove_stopwords(terms)
    word_delimiters = u'[\\[\\]\n.!?,;:\t\\-\\"\\(\\)\\\'\u2019\u2013 ]'
    term_list = re.split(word_delimiters, terms)
    trimmed = [x.rstrip() for x in term_list]
    stemmed = stem_list(trimmed)
    space = ' '
    normed = space.join(stemmed)
    normed = normed.replace('  ', ' ')
    return normed

In the code block above, we have functions to remove stopwords, stem, and normalize the text (remove special characters and trim white space). Apply the normalize function to every yelp review and assign the normalized text to a new column.

In [9]:
# apply normalized function
yelp['normalized_text'] = [normalize(text) for text in yelp['text']]

yelp.head()

,text,sentiment,normalized_text
0,Wow... Loved this place.,1,wow love place
1,Crust is not good.,0,crust not good
2,Not tasty and the texture was just nasty.,0,not tasti textur nasti
3,Stopped by during the late May bank holiday of...,1,stop late may bank holiday rick steve recommen...
4,The selection on the menu was great and so wer...,1,select menu great price


Next, use the one hot function for text encoding and encode the normalized text. Determine the vocabulary size to perform the encoding.

In [10]:
vocab = " ".join(yelp['normalized_text'].tolist())
vocab = vocab.split(" ")
vocab = list(filter(None, vocab))
vocab_size = len(set(vocab))
vocab[:20]

['wow',
 'love',
 'place',
 'crust',
 'not',
 'good',
 'not',
 'tasti',
 'textur',
 'nasti',
 'stop',
 'late',
 'may',
 'bank',
 'holiday',
 'rick',
 'steve',
 'recommend',
 'love',
 'it']

In [11]:
len(set(vocab))

1628

In [26]:
# Answer below:
vocab_size = len(set(vocab))
encoded_docs = [one_hot(doc, vocab_size) for doc in yelp['normalized_text']]


In [27]:
encoded_docs[:20]

[[246, 1567, 117],
 [1561, 215, 549],
 [215, 603, 1297, 1551],
 [464, 1043, 361, 89, 373, 422, 894, 94, 1567, 1337],
 [701, 48, 257, 1575],
 [953, 172, 595, 1131, 161],
 [786, 959, 135],
 [1447, 344, 1376, 678, 1502, 1073, 1159, 1605, 110, 1067],
 [679, 257, 1479],
 [257, 592],
 [1369, 1436],
 [1496, 215, 589, 672],
 [415, 787, 714, 231, 1058, 238, 53, 1393, 659],
 [216, 1083, 641, 1266, 1160, 1350, 891],
 [1266, 691, 1349, 1541, 128],
 [424, 787, 889, 1079, 32, 200],
 [546, 94],
 [414, 856, 1186, 1369],
 [117, 215, 1354, 1605, 205, 1087, 210],
 [215, 344, 756]]

Convert the encoded sequences into a numpy array and make sure all reviews are the same length using the `pad_sequences` function in Keras.

In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
# pad the encoded sequences
ind_vars = pad_sequences(encoded_docs)

ind_vars[:20]

array([[   0,    0,    0, ...,  246, 1567,  117],
       [   0,    0,    0, ..., 1561,  215,  549],
       [   0,    0,    0, ...,  603, 1297, 1551],
       ...,
       [   0,    0,    0, ...,  856, 1186, 1369],
       [   0,    0,    0, ...,  205, 1087,  210],
       [   0,    0,    0, ...,  215,  344,  756]], dtype=int32)

In [15]:
ind_vars.shape

(976, 83)

Split the data into train and test. Use 20% for test. The sentiment column should be used as the target variable.

In [16]:
from sklearn.model_selection import train_test_split

# define X and y
X = ind_vars
y = yelp['sentiment']

# test/train split our data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [17]:
max_words = np.max(X_train)
max_words

1627

In [18]:
X_train.shape

(780, 83)

In [19]:
y_train.shape

(780,)

In [20]:
X_test.shape

(196, 83)

In [21]:
y_test.shape

(196,)

Create a sequential model. The model should contain an embedding layer with input dim that is the size of the largest encoding in the vocabulary. The output dim should be 100, the input length is the number of columns in the training data. 
After the embedding layer, add a SimpleRNN layer with unit size 32, a dense layer of size 8 and a dense output layer.

In [28]:
import numpy as np
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN

# instantiate model
model = Sequential()
# embedding layer
model.add(Embedding(input_dim=max_words+1, output_dim=100, input_length=X_train.shape[1]))
# simple RNN layer
model.add(SimpleRNN(32))
# dense layer
model.add(Dense(8, activation='relu'))
# dense output layer
model.add(Dense(1, activation='sigmoid'))
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 83, 100)           162800    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                4256      
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 167,329
Trainable params: 167,329
Non-trainable params: 0
_________________________________________________________________


Compile using the optimizer of your choice, use crossentropy for your loss function. Fit the model using a batch size of 128 and 50 epochs

In [29]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=128)

Epoch 1/50
7/7 [==============================] - 2s 110ms/step - loss: 0.6953 - accuracy: 0.4833 - val_loss: 0.6825 - val_accuracy: 0.5765
Epoch 2/50
7/7 [==============================] - 1s 84ms/step - loss: 0.6559 - accuracy: 0.7210 - val_loss: 0.6763 - val_accuracy: 0.6071
Epoch 3/50
7/7 [==============================] - 1s 84ms/step - loss: 0.6058 - accuracy: 0.8302 - val_loss: 0.6614 - val_accuracy: 0.6020
Epoch 4/50
7/7 [==============================] - 1s 85ms/step - loss: 0.5688 - accuracy: 0.7459 - val_loss: 0.6563 - val_accuracy: 0.6429
Epoch 5/50
7/7 [==============================] - 1s 81ms/step - loss: 0.4976 - accuracy: 0.9220 - val_loss: 0.6362 - val_accuracy: 0.6837
Epoch 6/50
7/7 [==============================] - 0s 73ms/step - loss: 0.4342 - accuracy: 0.9460 - val_loss: 0.6194 - val_accuracy: 0.6990
Epoch 7/50
7/7 [==============================] - 1s 79ms/step - loss: 0.3633 - accuracy: 0.9784 - val_loss: 0.6130 - val_accuracy: 0.6990
Epoch 8/50
7/7 [==========